In [11]:
import pandas as pd
import numpy as np
import os

In [ ]:
model = 'geneva_200m'
params_file = 'swirl_03'
output_folder = r'../Outputs'

lvl0_filename = f'{model}_{params_file}_lvl0.csv'
output_filename = f'{model}_{params_file}_lvl1.csv'

In [4]:
level0_csv_path = os.path.join(output_folder, lvl0_filename)
level0_data = pd.read_csv(level0_csv_path)

In [ ]:
# Function to identify the different depths of an eddy
def identify_eddy(data, id_eddy, threshold):
    mask = (
        (data['date'] == data.at[id_eddy, 'date']) &
        (data['rotation_direction'] == data.at[id_eddy, 'rotation_direction']) &
        (np.sqrt((data['xc'] - data.at[id_eddy, 'xc'])**2 + (data['yc'] - data.at[id_eddy, 'yc'])**2) < threshold)
    )
    return data.loc[mask]

In [48]:
# Parameters
dist_threshold = 2  # in number of cells
eddy_rows_lvl1 = []  # Collect aggregated rows here
id_level1 = 0
idx_already_aggregated = set()

# Main loop
for idx in level0_data['id']:
    if idx in idx_already_aggregated:
        continue

    aggregated_data = identify_vortex(level0_data, idx, dist_threshold)

    row = {
        'id': id_level1,
        'id_lvl0': aggregated_data['id'].tolist(),
        'date': aggregated_data.at[idx, 'date'],
        'depth_min_[m]': aggregated_data['depth_[m]'].min(),
        'depth_max_[m]': aggregated_data['depth_[m]'].max(),
        'xc_mean': aggregated_data['xc'].mean(),
        'yc_mean': aggregated_data['yc'].mean(),
        'surface_area_mean_[m2]': aggregated_data['surface_area_[m2]'].mean(),
        'volume_[m3]': aggregated_data['volume_slice_[m3]'].sum(),
        'rotation_direction': aggregated_data.at[idx, 'rotation_direction'],
        'kinetic_energy_[MJ]': aggregated_data['kinetic_energy_[MJ]'].sum()
    }

    eddy_rows_lvl1.append(row)
    idx_already_aggregated.update(aggregated_data['id'].tolist())
    id_level1 += 1

# Create the final DataFrame using pd.concat
df_catalogue_level1 = pd.concat([pd.DataFrame([row]) for row in eddy_rows_lvl1], ignore_index=True)

In [52]:
df_catalogue_level1.to_csv(os.path.join(output_folder, output_filename), index=False)

TO DO 
- Check depth consistency